In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout,Activation

from keras.layers.convolutional import Conv1D
from keras import backend as K
from keras.layers.core import Lambda
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding

import keras
from keras.utils import np_utils

In [52]:
data = pd.read_csv("IDS_Combined_Data_IoT2.csv")

In [53]:
data.head(5)

,ts,date,time,fridge_temperature,temp_condition,label,type,door_state,sphone_signal,latitude,...,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,light_status,current_temperature,thermostat_status,temperature,pressure,humidity
0,1556245180,25-Apr-19,19:19:40,9.00,1,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,23.016184,1.035,46.343618
1,1556245180,25-Apr-19,19:19:40,9.25,1,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,38.620990,1.035,46.343618
2,1556245185,25-Apr-19,19:19:45,12.65,1,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,42.732741,1.035,46.343618
3,1556245185,25-Apr-19,19:19:45,4.65,0,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,37.785562,1.035,46.343618
4,1556245195,25-Apr-19,19:19:55,12.65,1,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,36.353584,1.035,46.343618


In [54]:
dataset = data.drop(['ts', 'date', 'time'], axis =1)

In [55]:
dataset.head(5)

,fridge_temperature,temp_condition,label,type,door_state,sphone_signal,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,light_status,current_temperature,thermostat_status,temperature,pressure,humidity
0,9.00,1,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,23.016184,1.035,46.343618
1,9.25,1,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,38.620990,1.035,46.343618
2,12.65,1,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,42.732741,1.035,46.343618
3,4.65,0,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,37.785562,1.035,46.343618
4,12.65,1,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,36.353584,1.035,46.343618


In [56]:
dataset['type'].value_counts()

normal        245000
backdoor       35000
injection      35000
password       35000
ddos           25000
ransomware     16030
xss             6116
scanning        3973
Name: type, dtype: int64

In [57]:
dataset.isna().sum().sum()

0

In [58]:
dataset.columns

Index(['fridge_temperature', 'temp_condition', 'label', 'type', 'door_state',
       'sphone_signal', 'latitude', 'longitude', 'FC1_Read_Input_Register',
       'FC2_Read_Discrete_Value', 'FC3_Read_Holding_Register', 'FC4_Read_Coil',
       'motion_status', 'light_status', 'current_temperature',
       'thermostat_status', 'temperature', 'pressure', 'humidity'],
      dtype='object')

In [59]:
# X = dataset.drop(['label','type'], axis = 1)
# X.shape

(401119, 17)

In [60]:
# y = dataset['type']
# y.shape

(401119,)

In [ ]:
from imblearn.over_sampling import SMOTE 
smote = SMOTE()
from collections import Counter

In [ ]:
XM_S = dataset.drop(['label', 'type'], axis = 1)
XM_S.shape

In [ ]:
yM_S = dataset['type'] # use type for 'multi', and use 'label for binary'
yM_S = yM_S.values
yM_S.shape

In [ ]:
XM, yM = smote.fit_resample(XM_S,yM_S)

In [ ]:
print("Before Smote :", Counter(yM_S))
print("After Smote :", Counter(yM))

In [ ]:
XM.shape

In [61]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8))) #don't change these values
X_T = sel.fit(XM)

In [62]:
sel_cols = X_T.get_support(indices=True)
X_VT = XM.iloc[:,sel_cols]
X_VT

,fridge_temperature,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,current_temperature,temperature,pressure,humidity
0,9.00,4.514077,14.421946,32450,32708,32035,32728,28.442693,23.016184,1.035000,46.343618
1,9.25,4.514077,14.421946,32450,32708,32035,32728,28.442693,38.620990,1.035000,46.343618
2,12.65,4.514077,14.421946,32450,32708,32035,32728,28.442693,42.732741,1.035000,46.343618
3,4.65,4.514077,14.421946,32450,32708,32035,32728,28.442693,37.785562,1.035000,46.343618
4,12.65,4.514077,14.421946,32450,32708,32035,32728,28.442693,36.353584,1.035000,46.343618
...,...,...,...,...,...,...,...,...,...,...,...
401114,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,24.250404,2.204924,37.024913
401115,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,20.155984,-2.030547,90.297894
401116,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,24.577566,0.872942,37.687701
401117,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,20.913710,3.168207,93.647950


In [63]:
scaler = MinMaxScaler()
X_M = scaler.fit_transform(X_VT)
print(X_M)

[[0.61538462 0.00821665 0.00811167 ... 0.10772138 0.53355618 0.46251056]
 [0.63461538 0.00821665 0.00811167 ... 0.66504042 0.53355618 0.46251056]
 [0.89615385 0.00821665 0.00811167 ... 0.81188986 0.53355618 0.46251056]
 ...
 [0.43846154 0.00821665 0.00811167 ... 0.16348547 0.52695586 0.3756012 ]
 [0.43846154 0.00821665 0.00811167 ... 0.03263243 0.62043775 0.93746771]
 [0.43846154 0.00821665 0.00811167 ... 0.04525255 0.58120503 0.3689465 ]]


In [64]:
trainX, testX, trainY, testY = train_test_split(X_M, yM, test_size = 0.20, random_state = 40)

In [65]:
print(X_M.shape)
print(y.shape)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(401119, 11)
(401119,)
(320895, 11)
(320895,)
(80224, 11)
(80224,)


In [68]:
#Test Train split
lr = LogisticRegression(multi_class='ovr') #for multiclass
lr.fit(trainX, trainY)
y_predict = lr.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

0.6575588352612685
              precision    recall  f1-score   support

    backdoor       0.00      0.00      0.00      7098
        ddos       0.33      0.23      0.27      5032
   injection       0.38      0.01      0.01      6930
      normal       0.73      0.97      0.83     48896
    password       0.39      0.52      0.44      7041
  ransomware       0.21      0.13      0.16      3154
    scanning       0.97      0.04      0.07       797
         xss       1.00      0.01      0.01      1276

    accuracy                           0.66     80224
   macro avg       0.50      0.24      0.22     80224
weighted avg       0.57      0.66      0.57     80224



In [ ]:
#Test Train split
lda = LinearDiscriminantAnalysis()
lda.fit(trainX, trainY)
y_predict = lda.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

In [ ]:
#Test Train split
#default parameters used in base paper (n_neighbors, default=5) (p, default=2 for Euclidean Distance)
knn = KNeighborsClassifier()
knn.fit(trainX, trainY)
y_predict = knn.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

In [ ]:
#Test Train split
rf = RandomForestClassifier(n_estimators=10,criterion='gini')
rf.fit(trainX, trainY)
y_predict1 = rf.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

In [67]:
#Test Train Split
dt = DecisionTreeClassifier(criterion='gini')
dt.fit(trainX, trainY)
y_predict1 = dt.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.7835934383725568
              precision    recall  f1-score   support

    backdoor       0.49      0.49      0.49      7098
        ddos       0.70      0.70      0.70      5032
   injection       0.67      0.68      0.68      6930
      normal       0.88      0.88      0.88     48896
    password       0.69      0.68      0.68      7041
  ransomware       0.66      0.66      0.66      3154
    scanning       0.73      0.72      0.72       797
         xss       0.62      0.61      0.62      1276

    accuracy                           0.78     80224
   macro avg       0.68      0.68      0.68     80224
weighted avg       0.78      0.78      0.78     80224



nb = GaussianNB()
nb.fit(trainX, trainY)
y_predict1 = nb.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

In [ ]:
#Test Train Split
svclassifier = SVC(kernel='rbf', gamma='auto')
svclassifier.fit(trainX, trainY)
y_predict = svclassifier.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

### LSTM

In [ ]:
print(X_M.shape)
print(y.shape)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
# LSTM training (label encoder + shape)
encoder = LabelEncoder()
encoder.fit(trainY)
y_train_L = encoder.transform(trainY) #for sparse_categorical
X_train_L = trainX.reshape((-1, 1, trainX.shape[1]))

In [ ]:
# LSTM training (label encoder + shape)
encoder = LabelEncoder()
encoder.fit(testY)
y_train_L = encoder.transform(testY) #for sparse_categorical
X_train_L = testX.reshape((-1, 1, testX.shape[1]))

In [69]:
#X_M.shape

(401119, 11)

In [70]:
#X1 = X_M.reshape((-1, 1, 11))

In [71]:
#X1.shape

(401119, 1, 11)

In [73]:
#len(np.unique(yM))

8

In [74]:
#encoder = LabelEncoder()
#encoder.fit(yM)
#y_E = encoder.transform(yM)

In [75]:
#y_E.shape

(401119,)

In [79]:
#np.unique(y_E)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [80]:
#(trainX, testX, trainY, testY) = train_test_split(X1, y_E, test_size = 0.20, random_state = 40)

In [81]:
# 1 LSTM Layer (input), 3 Dense Hidden Layers
model = Sequential()
model.add(LSTM(17, input_shape=(1, 11), activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(8,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 19s - loss: 0.7921 - accuracy: 0.7536 - val_loss: 0.6633 - val_accuracy: 0.8035
Epoch 2/35
5014/5014 - 16s - loss: 0.7082 - accuracy: 0.7824 - val_loss: 0.6471 - val_accuracy: 0.8022
Epoch 3/35


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

In [ ]:
model.summary()

### Conv-LSTM 

In [82]:
X_M.shape

(401119, 11)

In [83]:
X1 = X_M.reshape((-1, 1, 11))

In [84]:
X1.shape

(401119, 1, 11)

In [85]:
len(np.unique(yM))

8

In [86]:
encoder = LabelEncoder()
encoder.fit(yM)
y_E = encoder.transform(yM)

In [87]:
y_E.shape

(401119,)

In [90]:
(trainX, testX, trainY, testY) = train_test_split(X1, y_E, test_size = 0.20, random_state = 40)

In [91]:
nb_filter = 250
filter_length = 3

model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))
model.add(LSTM(17))
model.add(Dropout(0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 33s - loss: 0.7528 - accuracy: 0.7681 - val_loss: 0.6476 - val_accuracy: 0.8073
Epoch 2/35
5014/5014 - 30s - loss: 0.6868 - accuracy: 0.7914 - val_loss: 0.6371 - val_accuracy: 0.8111
Epoch 3/35


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

In [ ]:
model.summary()

IF Conv-LSTM doesn't achieve max accuracy then run this, other

In [89]:
nb_filter = 250
filter_length = 3

model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=1))
model.add(Activation('softmax'))
model.add(LSTM(17, return_sequences=True))
model.add(LSTM(17, return_sequences=True))
model.add(Dense(128, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(8,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 50s - loss: 0.7067 - accuracy: 0.7854 - val_loss: 0.6394 - val_accuracy: 0.8045
Epoch 2/35
5014/5014 - 47s - loss: 0.6363 - accuracy: 0.8062 - val_loss: 0.6273 - val_accuracy: 0.8083
Epoch 3/35


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

In [ ]:
model.summary()